## imports

In [1]:
import re
import json

alphanumeric='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.'
dot =  '('+('|'.join(alphanumeric))+')'

# print(dot)


# Regex to Postfix

In [2]:
# def validate_regex(regex):
#     try:
#        re.compile(regex)
#        return True
#     except:
#        return False 
    
def get_next_char(text, index):
    if index < len(text) - 1:
        return text[index + 1]
    else:
        
        return None

def get_previous_char(text, index):
    if index > 0:
        return text[index - 1]
    else:
        return None
    
def validate_regex(regex):
    stack = []
    opening_chars = "(["
    closing_chars = ")]"
    pair_map = {")": "(", "]": "["}
    is_range = False
    prev_char = None
    for index,char in enumerate(regex):
        if char.isalnum() or char  in (".+*?|-") or char in opening_chars or char in closing_chars :
                    if char in ("+") and ( get_previous_char(regex,index) is  None or  not  (get_previous_char(regex,index).isalnum() or get_previous_char(regex,index) in (')]*?.') )):
                        return False
                    if char in ("?") and ( get_previous_char(regex,index) is  None or  not  (get_previous_char(regex,index).isalnum() or get_previous_char(regex,index) in (')]*+.') )):
                        return False
                    if char in ("*") and ( get_previous_char(regex,index) is  None or  not  (get_previous_char(regex,index).isalnum() or get_previous_char(regex,index) in (')].') )):
                        return False
                    if char in ("|") and (( get_previous_char(regex,index)  is  None or  not  (get_previous_char(regex,index).isalnum() or get_previous_char(regex,index)  in ('.+*?)]'))) or (get_next_char(regex, index) is  None or  not  ( get_next_char(regex, index).isalnum() or get_next_char(regex, index)  in ('.+*?[(') ))):
                        return False
                    if is_range and prev_char is not None and prev_char != "-" and char != "-":
                        if char == "]" or ord(prev_char) >= ord(char) or prev_char == "[" or (prev_char.isupper() and not char.isupper()) or (prev_char.isdigit() and not char.isdigit()):
                            return False
                    prev_char=char
                    if char in opening_chars:
                        stack.append(char)
                    elif char in closing_chars:
                        if len(stack) == 0 or stack[-1] != pair_map[char]:
                            return False
                        stack.pop()
                    elif char == "-":
                        if stack[-1] != '[':
                            return False
                        is_range = True
                        prev_char = regex[index-1]
                    else:
                        is_range = False

                    if char in ("+*?|()") and (len(stack) >0 and stack[-1] == '['):
                        return False
                    if (char == ']' and get_previous_char(regex, index) == '[') or (char == ')' and get_previous_char(regex, index) == '('):
                        return False
        else :
            return False    
    return len(stack) == 0  



# def handle_dot(regex):
#    return re.sub(r'\.', dot, regex)

# def handle_range(regex):
#     new_regex = ""
#     replace = False
#     start = None
#     end= None
#     for i in range(len(regex)):
#         if regex[i] == '[':
#             new_regex += '('
#             replace =True
#         elif regex[i]== ']':
#              new_regex += ')'
#              replace =False
#         elif regex[i] == '-':
#             replace = True
#             start = alphanumeric.index(regex[i-1])
#             end = alphanumeric.index(regex[i+1])
#             for j in range(start+1,end):
#                 new_regex += alphanumeric[j]
#                 if(replace and regex[i+1] != ']'):
#                     new_regex+='|'
#         else:
#            new_regex += regex[i] 
#            if(replace and regex[i+1] != ']'):
#                     new_regex+='|'


#     return new_regex  


def concatenate(regex):
    new_regex = ""
    for i in range(len(regex)-1):
        new_regex += regex[i] 
        if regex[i] in ['*','+','?',')',']'] and regex[i+1] not in ['*','+','?',')',']', '#','|']:
            new_regex+="#"
        elif regex[i] in alphanumeric and  (regex[i+1] in alphanumeric or regex[i+1] in ['(','[']):
            new_regex+="#"
    
    new_regex += regex[-1]
    k=0
    new_var = ""
    while k <  len(new_regex):
        if new_regex[k] =='[':
            new_var += new_regex[k]
            while get_next_char(new_regex, k) != "]":
              k=k+1
              if new_regex[k] != "#":
                new_var += new_regex[k]
            k=k+1
            if new_regex[k] != "#":
             new_var += new_regex[k] 
            k=k+1 
            # print(new_var)
            # print(p[i]) 
        else: 
          new_var += new_regex[k]
          k=k+1
             


    return new_var

def infix_to_postfix(regex):
    queue = []
    stack = []
    operators={'(':0,"|":1, "#":2, "?":3, "+":4, "*":5}

    for i in range(len(regex)):
        if regex[i] == '(':
            stack.append(regex[i])
        elif regex[i] == ')':
            while stack[-1] != "(":  
                queue.append(stack.pop()) 
            stack.pop()     
        elif regex[i] in operators:
            while stack  and operators[regex[i]] <= operators[stack[-1]]:
                queue.append(stack.pop())
            stack.append(regex[i])

        else :
            queue.append(regex[i])    
    
    while stack:
        queue.append(stack.pop())

    return ''.join(queue  )  


def to_postfix(regex):
    if(validate_regex(regex)):
    #    regex = handle_dot(regex)
    #    print(regex)
    #    regex = handle_range(regex)
    #    print(regex)
       regex = concatenate(regex)
    #    print(regex)
       regex = infix_to_postfix(regex)
    #    print(regex)

       return regex


    


    

# print(handle_dot('ab.c'))
# print(handle_range("[a-z]+abc[Abc]*[1-5]"))
# print(concatenate("a+b*|c(5.f)"))
# print(infix_to_postfix("(A+.B*)?.(C|D)"))
print(to_postfix("."))        


.


# Postfix to NFA

In [3]:
class edge:
    def __init__(self):
       self.label=None
       self.destination=None


class state:
       def __init__(self):
        self.label=None
        self.outgoing_edges=[]




class NFA:    
       def __init__(self, start, accept,states):
        self.start_state = start
        self.acceptance_state = accept
        self.states = states
        

def construct_nfa (label, id, stack):
    start = state()
    acceptance = state()
    start.label = "S"+str(id)
    acceptance.label = "S"+str(id+1)

    Edge = edge()
    Edge.label = label
    Edge.destination = acceptance 
    start.outgoing_edges.append(Edge)

    result_nfa = NFA(start, acceptance, [start,acceptance])
    stack.append(result_nfa)
    #print(result_nfa.inner_states)

    return result_nfa, id+2


def nfa_concatination (stack):
   
    
    nfa2 = stack.pop()
    nfa1 = stack.pop()


    Edge = edge()
    Edge.label = 'ε'
    Edge.destination = nfa2.start_state
    nfa1.acceptance_state.outgoing_edges.append(Edge)

    #print(nfa1)
    #print(nfa1.inner_states)

    resultnfa = NFA(nfa1.start_state,  nfa2.acceptance_state, nfa1.states + nfa2.states)
    stack.append(resultnfa)
    return resultnfa


def nfa_or (stack, id):
  nfa1 = stack.pop()
  nfa2 = stack.pop()
  newStart = state()
  newStart.label= "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  Edge1 = edge()
  Edge1.label = 'ε'
  Edge1.destination = nfa1.start_state

  Edge2 = edge()
  Edge2.label = 'ε'
  Edge2.destination = nfa2.start_state

  newStart.outgoing_edges.append(Edge1)
  newStart.outgoing_edges.append(Edge2)


  Edge3 = edge()
  Edge3.label = 'ε'
  Edge3.destination = newEnd

  nfa1.acceptance_state.outgoing_edges.append(Edge3)

  Edge4 = edge()
  Edge4.label = 'ε'
  Edge4.destination = newEnd
  nfa2.acceptance_state.outgoing_edges.append(Edge4)


  result = NFA(newStart, newEnd, [newStart,newEnd]+ nfa1.states + nfa2.states)
  stack.append(result)
  return result, id+2



def nfa_zero_or_more (stack, id):
  nfa1 = stack.pop()



  newStart = state()
  newStart.label = "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  Edge1 = edge()
  Edge1.label = 'ε'
  Edge1.destination = newStart
  nfa1.acceptance_state.outgoing_edges.append(Edge1)



  Edge2 = edge()
  Edge2.label = 'ε'
  Edge2.destination=nfa1.start_state
  newStart.outgoing_edges.append(Edge2)

  Edge3 = edge()
  Edge3.label = 'ε'
  Edge3.destination = newEnd
  nfa1.acceptance_state.outgoing_edges.append(Edge3)



  Edge4 = edge()
  Edge4.label = 'ε'
  Edge4.destination = newEnd
  newStart.outgoing_edges.append(Edge4)
  
  result = NFA (newStart, newEnd, [newStart, newEnd] + nfa1.states)
  stack.append(result)
  return result, id+2


def nfa_one_or_more (stack, id):
   nfa1 = stack.pop()



   newStart = state()
   newStart.label = "S"+str(id)
   newEnd = state()
   newEnd.label = "S"+str(id+1)

   Edge1 = edge()
   Edge1.label ='ε'
   Edge1.destination = newStart
   nfa1.acceptance_state.outgoing_edges.append(Edge1)

   Edge2 = edge()
   Edge2.label='ε'
   Edge2.destination=nfa1.start_state
   newStart.outgoing_edges.append(Edge2)

   Edge3 = edge()
   Edge3.label='ε'
   Edge3.destination = newEnd
   nfa1.acceptance_state.outgoing_edges.append(Edge3)


   result = NFA (newStart, newEnd, [newStart, newEnd] + nfa1.states)
   stack.append(result)
   return result, id+2

def nfa_zero_or_one (stack, id):
  nfa1 = stack.pop()

  newStart = state()
  newStart.label = "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  Edge1 = edge()
  Edge1.label='ε'
  Edge1.destination=nfa1.start_state
  newStart.outgoing_edges.append(Edge1)

  Edge2 = edge()
  Edge2.label='ε'
  Edge2.destination = newEnd
  nfa1.acceptance_state.outgoing_edges.append(Edge2)

  
  Edge2 = edge()
  Edge2.label='ε'
  Edge2.destination = newEnd
  newStart.outgoing_edges.append(Edge2)



  result = NFA (newStart, newEnd, [newStart, newEnd] + nfa1.states)
  stack.append(result)
  return result, id+2

def postfix_to_nfa(postfix):
     stack = []
     id = 1
     i= 0
     while i < (len(postfix)):
        #  print("check",postfix[i])
         if postfix[i] in alphanumeric:
           _ ,id =  construct_nfa(postfix[i],id,stack)
           i=i+1
         elif postfix[i]=='[': 
            new_var = ''
            new_var += postfix[i]
            while get_next_char(postfix, i) != "]":
              i=i+1
              if postfix[i] != "#":
                new_var += postfix[i]
            i=i+1
            if postfix[i] != "#":
             new_var += postfix[i] 
            i=i+1 
            # print(new_var)
            # print(postfix[i]) 
            _ ,id =  construct_nfa(new_var,id,stack)   
 
         elif postfix[i] == '#':   
             nfa_concatination(stack) 
             i=i+1
         elif postfix[i] == '|':   
            _,id = nfa_or(stack,id)  
            i=i+1 
         elif  postfix[i] == '*':  
              _,id = nfa_zero_or_more(stack,id)  
              i=i+1
         elif  postfix[i] == '+':  
              _,id = nfa_one_or_more(stack,id)    
              i=i+1
         elif  postfix[i] == '?':  
               _,id = nfa_zero_or_one(stack,id) 
               i=i+1
         else:
              raise ValueError(f"an invialid operator is entered {postfix[i]}")   
              
     result = stack.pop()
     return result


def convert_nfa_to_dictionary(nfaobj):
  json_file = dict()
  json_file["startingState"] = nfaobj.start_state.label
  for state in nfaobj.states:
    state_dict = dict()
    if state == nfaobj.acceptance_state:
      state_dict["isTerminatingState"] = True
    else:
      state_dict["isTerminatingState"] = False
    for edg in state.outgoing_edges:
      if(edg.label in state_dict.keys()):
        state_dict[edg.label] = [] + [state_dict[edg.label]] + [edg.destination.label]
      else:
        state_dict[edg.label] = edg.destination.label
    json_file[state.label] = state_dict
  return json_file 

    










# render graph and json

In [4]:
from graphviz import Digraph

def save_json_file (dict,filename):
  
    json_object = json.dumps(dict, indent=6,ensure_ascii=False)
    with open(filename + ".json", "w", encoding="utf-8") as outfile:
        outfile.write(json_object)

def render_graph(graph,message,regex,filename, attr):
    gra = Digraph(graph_attr={'rankdir':'LR', 'bgcolor': attr["bgcolor"]})
    for state in graph:
        if state == "startingState":
            gra.node(state, style='invisible')
        else:
            shape = "doublecircle" if graph[state]["isTerminatingState"] else "circle"
            gra.node(state, shape=shape, style='filled', fillcolor= attr["node_fillcolor"])

    for from_state in graph:
        if from_state == "startingState":
            gra.edge(tail_name=from_state, head_name=graph["startingState"], color=attr["edge_color"])
            continue

        for input in graph[from_state]:
            if input == "isTerminatingState":
                continue
            to_states = graph[from_state][input]
            if type(to_states) == list:
                for to_state in to_states:
                    gra.edge(tail_name=from_state, head_name=to_state, label=input, color=attr["edge_color"])
            else:
                gra.edge(tail_name=from_state, head_name=to_states, label=input, color=attr["edge_color"])

    gra.attr(label=message + regex, fontcolor=attr["label_fontcolor"], fontsize=attr["label_fontsize"], fontname='bold')
    gra.format= 'png'
    gra.render(filename, view=True)

graph_attr = { 
        "bgcolor": "#ffffff",
        "node_fillcolor": "#ffffff",
        "edge_color": "#000000",
        "label_fontcolor": "black",
        "label_fontsize": "20"
        }        

In [5]:
regex="[a-zA-Z1-5]ab*"
# postfix = to_postfix(regex)
# nfa1 = postfix_to_nfa(postfix)
# nfa_dict = convert_nfa_to_dictionary(nfa1)
# save_json_file(nfa_dict,"nfa1")
# render_graph(nfa_dict,"NFA for ",regex,"nfa1", graph_attr)
# get_all_regex_data(regex)


# NFA to DFA

In [6]:
def get_epsilon_closure(nfa, input_state):

    stack = [input_state]
    closure = set()

    while stack:

        current_state = stack.pop()
        closure.add(current_state)

        if "ε" in nfa[current_state]:

            next_states = nfa[current_state]["ε"]
            if isinstance(next_states, str):
                next_states = [next_states]

            for state in next_states:
                if state not in closure:
                    stack.append(state)
                    closure.add(state)

    return tuple(sorted(closure))

def convert_nfa_to_dfa(nfa):

    nfa_start_state = nfa["startingState"]
    dfa_start_state = get_epsilon_closure(nfa,nfa_start_state)

    dfa = {}
    dfa["startingState"] = dfa_start_state

    stack = [dfa_start_state]

    while stack:
        current_state = stack.pop()
        dfa[current_state] = {}     #current_state = (s0,s1,s2) 
        dfa[current_state]["isTerminatingState"] = False

        for state in current_state:
            nfa_state = nfa[state]   # nfa_state -> dic of s0 in nfa

            for sub_key in nfa_state:

                if sub_key == "isTerminatingState":
                    dfa[current_state]["isTerminatingState"] |= nfa_state["isTerminatingState"]

                elif sub_key != "ε": # edges of s0
                    next_states = get_epsilon_closure(nfa,nfa_state[sub_key])
                    if sub_key in dfa[current_state]: # if edge already in dfa state
                        temp_set= set(dfa[current_state][sub_key])
                        temp_set.update(next_states)
                        dfa[current_state][sub_key] = tuple(sorted(temp_set))
                    else:
                        dfa[current_state][sub_key] = next_states
        
        for new_sub_key in dfa[current_state]:
            if new_sub_key != "isTerminatingState":
                new_added_state = dfa[current_state][new_sub_key]
                if new_added_state not in dfa:
                    stack.append(new_added_state)

    return dfa

def get_final_dfa(dfa):

    # create a dict for states with their new names
    new_states_names = {}
    for i,state in enumerate(dfa):
        if state != "startingState":
            new_states_names[state] = "S"+str(i-1)
    # print(new_states_names)

    # replace states with their new names
    new_dfa = dfa.copy()

    for key in dfa:
        if key != "startingState":

            for sub_key in dfa[key]:
                if sub_key !="isTerminatingState":
                    old_state = dfa[key][sub_key]  #('S1', 'S3')
                    new_state = new_states_names[old_state] #'S0'
                    new_dfa[key][sub_key] = new_state

            old_state = key  #('S1', 'S3')
            new_state = new_states_names[old_state] #'S0'
            new_dfa[new_state]= new_dfa[old_state]
            del new_dfa[key]

        else:
            old_state = dfa[key]
            new_state = new_states_names[old_state]
            new_dfa[key] = new_state

    return new_dfa

# Minimized DFA

In [7]:
def minimize_dfa(dfa):

    unique_inputs = set()

    groups_dic = {}
    groups_dic[1] = set()
    groups_dic[2] = set()

    for key in dfa:
        if key != "startingState":
            if dfa[key]["isTerminatingState"]:
                groups_dic[1].add(key)
            else:
                groups_dic[2].add(key)
        
            for sub_key in dfa[key]:
                if sub_key != "isTerminatingState":
                    unique_inputs.add(sub_key)

    if len(groups_dic[1]) == 0:
        del groups_dic[1]

    if len(groups_dic[2]) == 0:
        del groups_dic[2]

    # print(groups_dic,unique_inputs)

    #####################################################

    stop = False

    while not stop:

        stop = True

        groups_names = list(groups_dic.keys())
        
        for group_id in groups_names:
                        
            for input in unique_inputs:

                dic={}
                states_in_group = groups_dic[group_id] 
                # print('===========>',states_in_group,'=============>',input,'------------->',groups_dic)

                for state in states_in_group:
                    
                    if input in dfa[state]:
                        next_state = dfa[state][input]

                        for group, state_list in groups_dic.items():
                            if next_state in state_list:
                                if group in dic.keys():
                                    dic[group].add(state)
                                else:
                                    dic[group]={state}
                    else: 
                        if "-" in dic.keys():
                            dic["-"].add(state)
                        else:
                            dic["-"]={state}
                
                # print(dic) 

                if len(dic) > 1: # there is division
                    stop = False
                    i = max(groups_dic.keys()) + 1
                    del groups_dic[group_id]
                    for new_group in dic.keys():
                            groups_dic[i]=dic[new_group]
                            i+=1
                    break
            
                        
            
                    
    key_list = list(groups_dic.keys())
    new_dic = {}
    # print("groups dic",groups_dic)
    for key, value in groups_dic.items():
        index = key_list.index(key)
        value =  groups_dic[key]  # Get the value associated with the old key
        new_dic["S"+str(index)] = value  # Create a new key-value pair with the new key
        
       

        
    # print("groups dic",new_dic)
    inverted_dic = {}
    for key, value in new_dic.items():
        for val in value:
            inverted_dic[val]=key
    
    # inverted_dict = {tuple(value): key for key, value in new_dic.items()}
    minimized_dfa = {
        "startingState":  [key for key, value in new_dic.items() if dfa["startingState"] in value ][0]
    }
    # print(minimized_dfa)
    # print((dfa["startingState"]))
    # print(inverted_dic)
    # Build the minimized dfa in the standard json format specified
    for key, value in new_dic.items():
        # Let the first state in the group be its representative state
        group_first_state = list(value)[0]
        # Equate the transitions of the group with the transitions of its representative state
        minimized_dfa[key] = dfa[group_first_state]
        for inputt in minimized_dfa[key]:
            if inputt == "isTerminatingState":
                continue
            # Change the name of the destination states to the name of their corresponding groups
            minimized_dfa[key][inputt] = inverted_dic[minimized_dfa[key][inputt]]
    # print(minimized_dfa)
    return minimized_dfa                        
                    

In [8]:
test_for_min={'startingState': 'S0', 'S0': {'isTerminatingState': False, 'b': 'S4', 'c': 'S1'}, 'S1': {'isTerminatingState': False, 'b': 'S4', 'd': 'S2', 'c': 'S1'}, 'S2': {'isTerminatingState': True, 'c': 'S3'}, 'S3': {'isTerminatingState': False, 'd': 'S2'}, 'S4': {'isTerminatingState': False, 'b': 'S4','c':'S1'}}
try1 = {'startingState': 'S0', 'S0': {'isTerminatingState': False, 'a': 'S1'}, 'S1': {'isTerminatingState': False, 'a': 'S1', 'b': 'S2'}, 'S2': {'isTerminatingState': True}}
minimize_dfa (test_for_min)
# minimize_dfa (try1)
try2 = {'startingState': 'S0', 'S0': {'isTerminatingState': False, 'a': 'S1'}, 'S1': {'isTerminatingState': False, 'b': 'S2'}, 'S2': {'isTerminatingState': False, 'd': 'S5', 'b': 'S4', 'c': 'S3'}, 'S3': {'isTerminatingState': False, 'd': 'S5', 'b': 'S4', 'c': 'S3'}, 'S4': {'isTerminatingState': False, 'd': 'S5', 'b': 'S4', 'c': 'S3'}, 'S5': {'isTerminatingState': True, 'd': 'S5'}}
# minimize_dfa (try2)
# minimize_dfa (final_dfa)

In [9]:
test_for_min={'startingState': 'S0', 'S0': {'isTerminatingState': False, 'a': 'S6', 'b': 'S5', 'c': 'S1'}, 'S1': {'isTerminatingState': False, 'd': 'S4', 'e': 'S3', 'f': 'S2'}, 'S2': {'isTerminatingState': True}, 'S3': {'isTerminatingState': True}, 'S4': {'isTerminatingState': True}, 'S5': {'isTerminatingState': False, 'd': 'S4', 'e': 'S3', 'f': 'S2'}, 'S6': {'isTerminatingState': False, 'd': 'S4', 'e': 'S3','f':'S2'}}
minimize_dfa (test_for_min)

{'startingState': 'S2',
 'S0': {'isTerminatingState': True},
 'S1': {'isTerminatingState': False, 'd': 'S0', 'e': 'S0', 'f': 'S0'},
 'S2': {'isTerminatingState': False, 'a': 'S1', 'b': 'S1', 'c': 'S1'}}

In [10]:
def get_all_regex_data(regex):
    if(validate_regex(regex)):
        postfix = to_postfix(regex)
        nfa1 = postfix_to_nfa(postfix)
        nfa_dict = convert_nfa_to_dictionary(nfa1)
        save_json_file(nfa_dict,"nfa")
        render_graph(nfa_dict,"NFA for ",regex,"nfa", graph_attr)

        #dfa
        dfa1 = convert_nfa_to_dfa(nfa_dict)
        dfa1 = get_final_dfa (dfa1)
        # save_json_file(dfa1,"dfa")
        # render_graph(dfa1,"DFA for ",regex ,"dfa", graph_attr)

        #min dfa
        minimized_dfa1 = minimize_dfa (dfa1)
        save_json_file(minimized_dfa1,"minimized dfa")
        render_graph(minimized_dfa1,"minimized DFA for ",regex , "minimized dfa", graph_attr)
    else:
        print('sorry you have entered invalid regex! ')        

# test cases 

In [37]:
regex = input("Enter a regex: ")
get_all_regex_data(regex)